In [1]:
from model import *
from train_in_roll import *
from dataloader import *

In [2]:
dataset = ImageDataSet(win_size=20, start_date=20091231, end_date=20111231, mode='train', indicators={'MA':[20]}, show_volume=True).generate_images(0.04)

[Load Data] | 2023-04-15 15:53:51 Done | Using  6.408 seconds
TRAIN DataSet Initialized
 	 - Image Size:   (64, 60)
 	 - Time Period:  20091231 - 20111231
 	 - Indicators:   {'MA': [20]}
 	 - Volume Shown: True


Generating Images: 100%|██████████| 2283/2283 [00:41<00:00, 54.96it/s] 


In [3]:
#test_dataset = ImageDataSet(win_size=20, start_date=20111231, end_date=20121231, mode='train', indicators={'MA':[20]}, show_volume=True).generate_images(0.04)

In [4]:
train_len = int(len(dataset) * (1 - 0.2))
val_len = int(len(dataset)) - train_len
train_loader, val_loader = torch.utils.data.random_split(dataset, [train_len, val_len])
train_loader = torch.utils.data.DataLoader(dataset=train_loader, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_loader, batch_size=32, shuffle=False)
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [5]:
roll = [((20091231, 20111231), (20111231, 20121231))]
models = [CNN20d().to('cuda'),CNN20d().to('cuda'),CNN20d().to('cuda')]
train_model(models, train_loader, val_loader, num_epochs=10, learning_rate=0.001, batch_size=32, device='cuda' , weight_decay=0.0)

tensor([[[  0., 255.,   0.,  ...,   0., 255.,   0.],
         [  0., 255.,   0.,  ...,   0., 255.,   0.],
         [  0., 255.,   0.,  ...,   0., 255.,   0.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [255., 255.,   0.,  ...,   0.,   0.,   0.]],

        [[  0., 255.,   0.,  ...,   0., 255.,   0.],
         [  0., 255.,   0.,  ...,   0., 255.,   0.],
         [  0., 255.,   0.,  ...,   0., 255.,   0.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[  0., 255.,   0.,  ...,   0., 255.,   0.],
         [  0., 255.,   0.,  ...,   0., 255.,   0.],
         [  0., 255.,   0.,  ...,   0., 255.,   0.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [255., 255.,   0.,  ...,   0.,   0.,   0.]],

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x82944 and 46080x2)